In [31]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-

import cx_Oracle
import time
import os

def conndb(username, pwd, servername, ip, port,):
#定义数据库连接，需要输入三个参数
    username = username
    pwd = pwd
    tns = cx_Oracle.makedsn(ip, port, servername)
    db = cx_Oracle.connect(username,pwd,tns)
    return db

def insert_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    cursor.close()
    db.commit()

def insert_db_p(db, sql, para):
    cursor = db.cursor()
    cursor.execute(sql, para)
    cursor.close()
    db.commit()
    
def delete_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    cursor.close()
    
def select_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    cursor.close()
    return result

def recored(path, name, state):
    #f = open(r'C:\Users\yaol\Desktop\automatic_data_input.log', 'a')
    now = time.strftime('%Y-%m-%d_%H:%M:%S',time.localtime(time.time()))
    f = open(path, 'a')
    f.write(now + ' ' + name + state + '\n')
    f.close()

path = os.getcwd() + '\FAST入库记录.log'
#path = r'C:\Users\yaol\Desktop\automatic_data_input.log'

try:
    db = conndb('sqmdb', 'md00t', 'sqmmt', '192.156.8.95',1521)
#     tns = cx_Oracle.makedsn('192.156.8.95',1521,'sqmmt')
#     conn = cx_Oracle.connect('sqmdb','md00t',tns)

    print (db.version)
    recored(path, 'connect to fast database', ' =====>  Success')

    # 提取系统时间 用于重组表名
    select_sql = "select to_char(sysdate - interval '1' day,'yyyymmdd') from dual"
    row = select_db(db, select_sql)
    date_msg = '待入库数据日期：' +  row[0][0]
    print(date_msg)
    recored(path, date_msg, ' ')
    #重组table name
    for h in range(0, 24):
        n = str(h)
        s = n.zfill(2)  # 保证输出的数字为两位
        for dblink in range(1, 3):
            t_table = row[0][0] + s + '@do_dblink' + str(dblink) + ' t'  # 重组table name 格式如：2017091921@do_dblink1 t
###########################################################
            DO_S1MSGLATENCY = '''INSERT INTO DO_S1MSGLATENCY 
        select /*+ DRIVING_SITE(t) parallel(t 4)*/TRUNC(TS,'HH24') SDATE,'SHENYANG' CITY,ts, imsi, ci,floor(substr(ci,6)/256) enbid,
        mod(substr(ci,6),256) cellid, decoderid, mme_ip, enb_ip, enbues1apid, mmeues1apid, tac, mtmsi,
        imei, msg, latency, timeout, retrans, radionetworklayercause, transportlayercause, nascause, protocolcause,
        misccause, nasesmcause, nasemmcause, succflag,1 SYSTEM_ID
        from s1msglatency_{}; '''.format(t_table)
###########################################################

###########################################################
            DO_S1DSS2 = '''INSERT INTO DO_S1DSS2
select * from (select TRUNC(TS_START,'HH24') SDATE,'SHENYANG' CITY,MS_IMEI,IMSI,CI,BEARLAYERTYPE
,SERVICETYPE,TS_START,URI,URI_PATH,USERAGENT,USERAGENTEXT,CONTENTTYPE,PAYLOAD_DL,PAYLOAD_UL,NULL LAT,NULL LON,1 SYSTEM_ID
from s1dss2_{}
where IMSI >0
) a;'''.format(t_table)
###########################################################
            try:
                print("执行 =====>  ", 's1msglatency_' + t_table)
                insert_db(db, DO_S1MSGLATENCY)
                s1msg = 's1msglatency_' + t_table
                recored(path, s1msg, ' =====>  Success')
                print("执行 =====>  ", 's1dss2_' + t_table)
                insert_db(db, DO_S1DSS2)
                s1dss2 = 's1dss2_' + t_table
                recored(path, s1dss2, ' =====>  Success')
            except:
                recored(path, t_table, ' =====>  Fail')

except:
    recored(path, 'connect to fast database', ' =====>  Fail')
recored(path, 'The program has been executed!', ' ')


11.2.0.4.0
待入库数据日期：20170920
执行 =====>   s1msglatency_2017092000@do_dblink1 t
执行 =====>   s1msglatency_2017092000@do_dblink2 t
执行 =====>   s1msglatency_2017092001@do_dblink1 t
执行 =====>   s1msglatency_2017092001@do_dblink2 t
执行 =====>   s1msglatency_2017092002@do_dblink1 t
执行 =====>   s1msglatency_2017092002@do_dblink2 t
执行 =====>   s1msglatency_2017092003@do_dblink1 t
执行 =====>   s1msglatency_2017092003@do_dblink2 t
执行 =====>   s1msglatency_2017092004@do_dblink1 t
执行 =====>   s1msglatency_2017092004@do_dblink2 t
执行 =====>   s1msglatency_2017092005@do_dblink1 t
执行 =====>   s1msglatency_2017092005@do_dblink2 t
执行 =====>   s1msglatency_2017092006@do_dblink1 t
执行 =====>   s1msglatency_2017092006@do_dblink2 t
执行 =====>   s1msglatency_2017092007@do_dblink1 t
执行 =====>   s1msglatency_2017092007@do_dblink2 t
执行 =====>   s1msglatency_2017092008@do_dblink1 t
执行 =====>   s1msglatency_2017092008@do_dblink2 t
执行 =====>   s1msglatency_2017092009@do_dblink1 t
执行 =====>   s1msglatency_2017092009@do_db

In [24]:
print (time.strftime('%Y-%m-%d_%H:%M:%S',time.localtime(time.time())))
print (time.localtime(time.time()))

2017-09-20_17:18:59
time.struct_time(tm_year=2017, tm_mon=9, tm_mday=20, tm_hour=17, tm_min=18, tm_sec=59, tm_wday=2, tm_yday=263, tm_isdst=0)


In [6]:
import os
pwd = os.getcwd() + '\FAST入库记录.log'
print(pwd)

C:\Users\yaol\FAST入库记录.log


11.2.0.4.0
select to_char(sysdate - interval '1' day,'yyyymmdd') from dual


In [11]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-

import cx_Oracle
import time
import os

def conndb(username, pwd, servername, ip, port,):
#定义数据库连接，需要输入三个参数
    username = username
    pwd = pwd
    tns = cx_Oracle.makedsn(ip, port, servername)
    db = cx_Oracle.connect(username,pwd,tns)
    return db

def insert_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    cursor.close()
    db.commit()

def insert_db_p(db, sql, para):
    cursor = db.cursor()
    cursor.execute(sql, para)
    cursor.close()
    db.commit()
    
def delete_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    cursor.close()
    
def select_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    cursor.close()
    return result

def recored(path, name, state):
    #f = open(r'C:\Users\yaol\Desktop\automatic_data_input.log', 'a')
    now = time.strftime('%Y-%m-%d_%H:%M:%S',time.localtime(time.time()))
    f = open(path, 'a')
    f.write(now + ' ' + name + state + '\n')
    f.close()

path = os.getcwd() + '\FAST入库记录.log'
#path = r'C:\Users\yaol\Desktop\automatic_data_input.log'


In [12]:
db = conndb('sqmdb', 'md00t', 'sqmmt', '192.156.8.95',1521)
#     tns = cx_Oracle.makedsn('192.156.8.95',1521,'sqmmt')
#     conn = cx_Oracle.connect('sqmdb','md00t',tns)

print (db.version)
recored(path, 'connect to fast database', ' =====>  Success')

11.2.0.4.0


In [13]:
select_sql = "select to_char(sysdate - interval '1' day,'yyyymmdd') from dual"
row = select_db(db, select_sql)
date_msg = '待入库数据日期：' +  row[0][0]
print(date_msg)
recored(path, date_msg, ' ')

待入库数据日期：20170924


In [18]:
for h in range(0, 24):
    n = str(h)
    s = n.zfill(2)  # 保证输出的数字为两位
    for dblink in range(1, 3):
        t_table = row[0][0] + s + '@do_dblink' + str(dblink) + ' t'  # 重组table name 格式如：2017091921@do_dblink1 t

###########################################################
#         DO_S1DSS2 = '''INSERT INTO yaoliang
# select * from (select TRUNC(TS_START,'HH24') SDATE,'SHENYANG' CITY,MS_IMEI,IMSI,CI,BEARLAYERTYPE
# ,SERVICETYPE,TS_START,URI,URI_PATH,USERAGENT,USERAGENTEXT,CONTENTTYPE,PAYLOAD_DL,PAYLOAD_UL,NULL LAT,NULL LON,1 SYSTEM_ID
# from s1dss2_{}
# where IMSI >0 and rownum < 10
# ) a;'''.format(t_table)
        DO_S1DSS2 = """INSERT INTO yaoliang select * from (select TRUNC(TS_START,'HH24') SDATE, 'SHENYANG' CITY, MS_IMEI,IMSI,CI,BEARLAYERTYPE,
        SERVICETYPE,TS_START,URI,URI_PATH,USERAGENT,USERAGENTEXT,CONTENTTYPE,PAYLOAD_DL,PAYLOAD_UL,NULL LAT,NULL LON, 1 SYSTEM_ID FROM 
        S1DSS2_{} WHERE IMSI>0 AND ROWNUM < 10) a""".format(t_table)
###########################################################
        print(DO_S1DSS2)
        cursor = db.cursor()
        cursor.execute(DO_S1DSS2)
        cursor.close()
        db.commit()        

#         print("执行 =====>  ", 's1dss2_' + t_table)
#         insert_db(db, DO_S1DSS2)
#         s1dss2 = 's1dss2_' + t_table
#         recored(path, s1dss2, ' =====>  Success')

INSERT INTO yaoliang select * from (select TRUNC(TS_START,'HH24') SDATE, 'SHENYANG' CITY, MS_IMEI,IMSI,CI,BEARLAYERTYPE,
        SERVICETYPE,TS_START,URI,URI_PATH,USERAGENT,USERAGENTEXT,CONTENTTYPE,PAYLOAD_DL,PAYLOAD_UL,NULL LAT,NULL LON, 1 SYSTEM_ID FROM 
        S1DSS2_2017092400@do_dblink1 t WHERE IMSI>0 AND ROWNUM < 10) a;


DatabaseError: ORA-00911: invalid character

In [2]:
#! /usr/bin/env python
# -*- coding: utf-8 -*-

import cx_Oracle
import time
import os

def conndb(username, pwd, servername, ip, port,):
#定义数据库连接，需要输入三个参数
    username = username
    pwd = pwd
    tns = cx_Oracle.makedsn(ip, port, servername)
    db = cx_Oracle.connect(username,pwd,tns)
    return db

def insert_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    cursor.close()
    db.commit()

def insert_db_p(db, sql, para):
    cursor = db.cursor()
    cursor.execute(sql, para)
    cursor.close()
    db.commit()
    
def delete_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    cursor.close()
    
def select_db(db, sql):
    cursor = db.cursor()
    cursor.execute(sql)
    result = cursor.fetchall()
    cursor.close()
    return result

def recored(path, name, state):
    #f = open(r'C:\Users\yaol\Desktop\automatic_data_input.log', 'a')
    now = time.strftime('%Y-%m-%d_%H:%M:%S',time.localtime(time.time()))
    f = open(path, 'a')
    f.write(now + ' ' + name + state + '\n')
    f.close()

path = os.getcwd() + '\FAST入库记录.log'
#path = r'C:\Users\yaol\Desktop\automatic_data_input.log'

try:
#    db = conndb('sqmdb', 'md00t', 'sqmmt', '192.156.8.95',1521)
    db = conndb('fast', 'F@st*123', 'FAST', '192.156.17.197',1521)
#     tns = cx_Oracle.makedsn('192.156.8.95',1521,'sqmmt')
#     conn = cx_Oracle.connect('sqmdb','md00t',tns)

#    print (db.version)
    recored(path, 'connect to fast database', ' =====>  Success')

    # 提取系统时间 用于重组表名
    select_sql = "select to_char(sysdate - interval '1' day,'yyyymmdd') from dual"
    row = select_db(db, select_sql)
    date_msg = '待入库数据日期：' +  row[0][0]
#    print(date_msg)
    recored(path, date_msg, ' ')
    #重组table name
    for h in range(0, 24):
        n = str(h)
        s = n.zfill(2)  # 保证输出的数字为两位
        for dblink in range(1, 3):
            t_table = row[0][0] + s + '@do_dblink' + str(dblink) + ' t'  # 重组table name 格式如：2017091921@do_dblink1 t
###########################################################
            DO_S1MSGLATENCY = '''INSERT INTO DO_S1MSGLATENCY 
        select /*+ DRIVING_SITE(t) parallel(t 4)*/TRUNC(TS,'HH24') SDATE,'SHENYANG' CITY,ts, imsi, ci,floor(substr(ci,6)/256) enbid,
        mod(substr(ci,6),256) cellid, decoderid, mme_ip, enb_ip, enbues1apid, mmeues1apid, tac, mtmsi,
        imei, msg, latency, timeout, retrans, radionetworklayercause, transportlayercause, nascause, protocolcause,
        misccause, nasesmcause, nasemmcause, succflag,1 SYSTEM_ID
        from s1msglatency_{} ;'''.format(t_table)
###########################################################

###########################################################
            DO_S1DSS2 = '''INSERT INTO DO_S1DSS2
select * from (select TRUNC(TS_START,'HH24') SDATE,'SHENYANG' CITY,MS_IMEI,IMSI,CI,BEARLAYERTYPE
,SERVICETYPE,TS_START,URI,URI_PATH,USERAGENT,USERAGENTEXT,CONTENTTYPE,PAYLOAD_DL,PAYLOAD_UL,NULL LAT,NULL LON,1 SYSTEM_ID
from s1dss2_{}
where IMSI >0
) a'''.format(t_table)
###########################################################
            try:
                print("执行 =====>  ", 's1msglatency_' + t_table)
                insert_db(db, DO_S1MSGLATENCY)
                s1msg = 's1msglatency_' + t_table
                recored(path, s1msg, ' =====>  Success')

            except cx_Oracle.DatabaseError as msg:
                recored(path, t_table, ' =====>  Fail  Error msg: {}'.format(msg))
#            try:
#                print("执行 =====>  ", 's1dss2_' + t_table)
#                print (DO_S1DSS2)
#                insert_db(db, DO_S1DSS2)
#                s1dss2 = 's1dss2_' + t_table
#                recored(path, s1dss2, ' =====>  Success')
#            except:
#                recored(path, t_table, ' =====>  Fail')
                
    count_sql_s1dss2 = "select count(*) from DO_S1DSS2 WHERE sdate >= to_date('{}','yyyy-mm-dd') ".format(row[0][0])
    count_sql_s1msg = "select count(*) from DO_S1DSS2 WHERE sdate >= to_date('{}','yyyy-mm-dd') ".format(row[0][0])
    try:
        insert_count = select_db(db, count_sql_s1msg)
        recored(path, '插入成功条数：',str(insert_count[0][0]))
        insert_count = select_db(db, count_sql_s1dss2)
        recored(path, '插入成功条数：',str(insert_count[0][0]))
    except:
        recored(path, '插入数据量提取失败', ' ')
 

except:
    recored(path, 'connect to fast database', ' =====>  Fail')
recored(path, 'The program has been executed!', ' ')


执行 =====>   s1msglatency_2017092700@do_dblink1 t
执行 =====>   s1msglatency_2017092700@do_dblink2 t
执行 =====>   s1msglatency_2017092701@do_dblink1 t
执行 =====>   s1msglatency_2017092701@do_dblink2 t
执行 =====>   s1msglatency_2017092702@do_dblink1 t
执行 =====>   s1msglatency_2017092702@do_dblink2 t
执行 =====>   s1msglatency_2017092703@do_dblink1 t
执行 =====>   s1msglatency_2017092703@do_dblink2 t
执行 =====>   s1msglatency_2017092704@do_dblink1 t
执行 =====>   s1msglatency_2017092704@do_dblink2 t
执行 =====>   s1msglatency_2017092705@do_dblink1 t
执行 =====>   s1msglatency_2017092705@do_dblink2 t
执行 =====>   s1msglatency_2017092706@do_dblink1 t
执行 =====>   s1msglatency_2017092706@do_dblink2 t
执行 =====>   s1msglatency_2017092707@do_dblink1 t
执行 =====>   s1msglatency_2017092707@do_dblink2 t
执行 =====>   s1msglatency_2017092708@do_dblink1 t
执行 =====>   s1msglatency_2017092708@do_dblink2 t
执行 =====>   s1msglatency_2017092709@do_dblink1 t
执行 =====>   s1msglatency_2017092709@do_dblink2 t
执行 =====>   s1msglat

In [1]:
import time

print (time.time())

1506775166.1483548


In [2]:
print(time.gmtime())

time.struct_time(tm_year=2017, tm_mon=9, tm_mday=30, tm_hour=12, tm_min=39, tm_sec=35, tm_wday=5, tm_yday=273, tm_isdst=0)


In [3]:
time.asctime(time.gmtime())

'Sat Sep 30 12:39:51 2017'

In [4]:
time.localtime()

time.struct_time(tm_year=2017, tm_mon=9, tm_mday=30, tm_hour=20, tm_min=40, tm_sec=1, tm_wday=5, tm_yday=273, tm_isdst=0)

In [6]:
time.strftime('%Y%m%d %H:%M%S+%Z')

'20170930 20:4101+China Standard Time'

In [9]:
time.strftime('%Y-%m-%d')

'2017-09-30'

In [10]:
print('''fsafsa_{}'''.format(time.strftime('%Y-%m-%d')))

fsafsa_2017-09-30


In [11]:
import datetime

datetime.date.today()

datetime.date(2017, 9, 30)

In [15]:
today = datetime.date.today()
print(today.replace()(day = today.day - 1).strftime('%Y-%m-%d'))

TypeError: 'datetime.date' object is not callable

In [1]:
import datetime  
print (datetime.date.today())  
print (datetime.datetime.now().time() )
print (datetime.datetime.now() ) 
  
today = datetime.date.today()  
next_year = today.replace(day = today.day - 1).strftime("%Y.%m.%d")  
print  (next_year ) 
  
print (datetime.date(2000,2,29)-datetime.date(2001,2,28)  )

2017-10-05
21:04:58.098070
2017-10-05 21:04:58.098070
2017.10.04
-365 days, 0:00:00


In [2]:
print(today.day)

1


In [3]:
print(today-datetime.timedelta(days=1))

2017-09-30


In [4]:
print(today+datetime.timedelta(days=1))

2017-10-02


In [5]:
print("""afdjl_{}""".format(today-datetime.timedelta(days=1)))

afdjl_2017-09-30


In [9]:
input_day = today-datetime.timedelta(days=1)

SHIYI = '''MERGE INTO travel_datebase_shiyi shiyi USING 
    (select to_char(a.t,'yyyy-mm-dd') sdate,a.imsi,b.area,
    case when b.property_1 is null then 99 else b.property_1 end property_1,
     case  when b.property_2 is null then 99 else b.property_2 end property_2
    from do_s1message_lncuc_temp a,travel_datebase_yaoliang b 
    where 
    a.ci = b.eci and 
    a.t >= to_date('{}','yyyy-mm-dd')
    group by  to_char(a.t,'yyyy-mm-dd'),a.imsi,b.area,b.property_1,b.property_2) c on 
    (shiyi.sdate = c.sdate and
    shiyi.imsi = c.imsi and
    shiyi.area = c.area and
    shiyi.property_1 = c.property_1 and
    shiyi.property_2 = c.property_2) when not matched then 
    insert (shiyi.sdate,shiyi.imsi,shiyi.area,shiyi.property_1,shiyi.property_2) 
    values (c.sdate,c.imsi,c.area,c.property_1,c.property_2) '''.format(input_day)

print(SHIYI)

MERGE INTO travel_datebase_shiyi shiyi USING 
    (select to_char(a.t,'yyyy-mm-dd') sdate,a.imsi,b.area,
    case when b.property_1 is null then 99 else b.property_1 end property_1,
     case  when b.property_2 is null then 99 else b.property_2 end property_2
    from do_s1message_lncuc_temp a,travel_datebase_yaoliang b 
    where 
    a.ci = b.eci and 
    a.t >= to_date('2017-09-30','yyyy-mm-dd')
    group by  to_char(a.t,'yyyy-mm-dd'),a.imsi,b.area,b.property_1,b.property_2) c on 
    (shiyi.sdate = c.sdate and
    shiyi.imsi = c.imsi and
    shiyi.area = c.area and
    shiyi.property_1 = c.property_1 and
    shiyi.property_2 = c.property_2) when not matched then 
    insert (shiyi.sdate,shiyi.imsi,shiyi.area,shiyi.property_1,shiyi.property_2) 
    values (c.sdate,c.imsi,c.area,c.property_1,c.property_2) 


In [2]:
abc = ['2017100116','2017100121','2017100208','2017100208','2017100209','2017100212','2017100213','2017100214',
'2017100216','2017100217','2017100218','2017100221','2017100222']

for aaa in abc:
    print(aaa, '',str(int(aaa)+1))

2017100116  2017100117
2017100121  2017100122
2017100208  2017100209
2017100208  2017100209
2017100209  2017100210
2017100212  2017100213
2017100213  2017100214
2017100214  2017100215
2017100216  2017100217
2017100217  2017100218
2017100218  2017100219
2017100221  2017100222
2017100222  2017100223


In [6]:
bbb = input('请输入需要补采的日期，格式为YYYYMMDDHH（多个时段使用英文逗号隔开）：')
aaa = []
for x in bbb.split(','):
    aaa.append(x)
print(aaa)

请输入需要补采的日期，格式为YYYYMMDDHH（多个时段使用英文逗号隔开）：123
['123']
